### Summary
Record mean 1 row of the csv file. 

This notebook converts the records into alarms. 

In [1]:
import pandas as pd
import gc # for forcing garbage collection
import time
from alarms import  convertRecordsToAlarms
start = time.time()

In [2]:
input_fname = "formatted-pre-1-months4.csv" 
output_fname = "formatted-pre-2-" + input_fname.split("-")[-1]
path = "./data/new/"
output_fname

'formatted-pre-2-months4.csv'

In [3]:
df = pd.read_csv(path + input_fname, low_memory=False, parse_dates=["EventTime"])

# only for testing
# # # df = df.loc[df['SourceName'].isin(["48TIC2040"])]
# # # df.to_csv(path+"test.csv",index = False)
# # # df = pd.read_csv(path + "test.csv", low_memory=False, parse_dates=["EventTime"])

df.info()
# df["EventTime"] = df["EventTime"].apply(lambda d: parse(d))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5505780 entries, 0 to 5505779
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   EventTime    datetime64[ns]
 3   Message      object        
 4   Quality      int64         
 5   Condition    object        
 6   Mask         int64         
 7   NewState     int64         
 8   Status       int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 378.1+ MB


In [4]:
df

,MachineName,SourceName,EventTime,Message,Quality,Condition,Mask,NewState,Status
0,PHD47B,48FIX2040,2019-03-11 05:09:04,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IO...,192,IOP-,193,1,1
1,PHD47B,47TI931A,2019-03-11 05:09:24,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C I...,192,IOP,201,1,1
2,PHD47B,48TIC2027,2019-03-11 05:09:36,48TIC2027 E209 CIKIS REJEN DRAM PV = -6.0 C IO...,192,IOP-,201,1,1
3,PHD47B,47TI931A,2019-03-11 05:08:54,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C I...,192,IOP,201,1,1
4,PHD47B,47TI931A,2019-03-11 05:08:36,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C I...,192,IOP,201,1,1
...,...,...,...,...,...,...,...,...,...
5505775,PHD47B,47TI931A,2019-06-20 22:55:55,47TI931A C5 1.YTK GIRDISCAP SICAK PV = 691 C IOP,0,IOP,201,3,1
5505776,PHD47B,48TIC2026,2019-06-20 22:58:52,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.4 C VEL-,192,VEL-,193,3,1
5505777,PHD47B,48TIC2026,2019-06-20 22:58:57,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.3 C VEL-,192,VEL-,193,3,1
5505778,PHD47B,48TIC2026,2019-06-20 22:59:21,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.0 C VEL+,192,VEL+,193,3,1


In [5]:
alarms = []
print("Total Number of Sources :  ", len(df["SourceName"].unique()))
print("Staring Processing records of each source : ", end= " ")
index = 0 # only for printing 
for sname in df["SourceName"].unique():
    records =  df.loc[df['SourceName'].isin([sname])]
    records = records.to_dict(orient="records")
#     convertRecordsToAlarms()
    temp_alarms = convertRecordsToAlarms(records)
    print("({},{},{})".format(index,sname, len(temp_alarms)), end=" ")
    alarms += temp_alarms # concat list
    index += 1
    
#     if index == 2: # just for debugging
#         break
print("\n\n Done: All records are converted to to alarms.")

Total Number of Sources :   1000


Staring Processing records of each source :  (0,48FIX2040,113926) (1,47TI931A,894699) (2,48TIC2027,113924) (3,47PI1734,27616) (4,48PI2025,24022) (5,47TI1713,20432) (6,47LIC3103,470) (7,47LIC3408,256) (8,48XL002-ANN,1870) (9,SIO-18XA001,1954) (10,47TI2111A,150) (11,47TIC3520,981) (12,47TI2111B,94) (13,47FLI-039,2024) (14,47LIC3409,136) (15,48AI004,5383) (16,48AAHH004-ANN,5399) (17,47LI024,465) (18,47TI1512B,468) (19,47TI1512A,514) (20,47TI1512ABC,453) (21,48FI2003A,662) (22,48FALL2003-ANN,343) (23,47TI2112B,84) (24,47TI002C,33998) (25,47FIC1522A,481) (26,47TXI1592,99) (27,47LIC005,3748) (28,47TI1512C,409) (29,48FFIC2002,3164) (30,48XL012-ANN,111) (31,47UA2500-ANN,172) (32,47PIC027-ANN,587) (33,47PIC2028,92) (34,47TI870D,180) (35,47PDI003,805) (36,47XL016-ANN,641) (37,47XL015-ANN,648) (38,47FSL018-ANN,1) (39,47LI003,1329) (40,47FIC1520,132) (41,47TI003B,24414) (42,47PIC1505,717) (43,47TI2077,164) (44,47TI2076,20) (45,47TI2075,7) (46,47TI2073,12) (47,19LI105A,107444) (48,47TIC2078,92) (49

(424,47TIC3417,32) (425,47FIC1563,76) (426,47FIC013,27) (427,47FIC2007,4) (428,47TI618,38) (429,47XL1525A-ANN,11) (430,47LAL603-ANN,20) (431,47LAH603-ANN,12) (432,47TI3433,6) (433,48UA012-ANN,43) (434,47APC_NS_OFF-AN,5) (435,47PIC1507,6) (436,47APC_NS_WD-ANN,4) (437,47XL011-ANN,9) (438,47PI1707,27) (439,47APC_FRC_WD-ANN,8) (440,47TI1546,13) (441,47G17B-ANN,5) (442,47LIC1515,24) (443,47TI414E,7) (444,47PIC1713,10) (445,47LIC1514,46) (446,47XL1557B-ANN,10) (447,47TI412,239) (448,48PALL001X-ANN,3) (449,48FIC007,8) (450,47FLI-033,2) (451,47FD-033F-ANN,1) (452,01XA049-ANN,1) (453,48HA012-ANN,85) (454,47XL3602-ANN,3) (455,48XL004-ANN,4) (456,48HA005-ANN,5) (457,47HSD-006H-ANN,5) (458,48FI006A,4) (459,47HSD-005H-ANN,12) (460,48FI006B,4) (461,47AAH301-ANN,3) (462,47TAH418A-ANN,39) (463,TESTPIC027,11) (464,47TI2094,14) (465,48HA031-ANN,1) (466,TEST_FIC2065,3) (467,47HSD-005HH-ANN,4) (468,47XL3109-ANN,2) (469,47TI2111,1) (470,47PI1512,57) (471,47FIC1535,26) (472,47PI2019,1) (473,47HCD-004H-ANN,1

(858,47TI2049,2) (859,47PAL762-ANN,2) (860,47TI1731,5) (861,47TI1574,5) (862,47TI1701,2) (863,47FIC1540,5) (864,47TI1761,1) (865,47TI1729,5) (866,47TI1543,1) (867,47TI1518,1) (868,47TI1539,1) (869,47VAH600-ANN,1) (870,47TI1544,2) (871,47UA753C-ANN,2) (872,47XL751-ANN,0) (873,47XL020-ANN,0) (874,47TI1707,3) (875,47TI1551,2) (876,47XL1557D-ANN,5) (877,47TI2032,1) (878,47TI1719,1) (879,47TI3103,2) (880,47UA753B-ANN,2) (881,47FALL801-ANN,2) (882,47LI1558,1) (883,47G23-ANN,0) (884,47PI1512D,1) (885,47PI1512C,1) (886,47PI1512B,1) (887,47PI2067,1) (888,47TI803D,1) (889,47ZAL1513,0) (890,47TI803A,1) (891,47TI3310,1) (892,47PIC1712,2) (893,47PI1709,3) (894,47F101CND02,1) (895,47TI1762,2) (896,47TI803C,1) (897,47TI803B,1) (898,47XL3607-ANN,1) (899,47PI860,4) (900,47G108C-RUN-ANN,1) (901,47TIC014,0) (902,47TI616,5) (903,47G121B-ANN,1) (904,47TI860,1) (905,47UA753D-ANN,0) (906,47FD-066F-ANN,6) (907,47G121A-ANN,2) (908,47PI2148ABC,3) (909,47XL1565A-ANN,3) (910,47XL017A-ANN,2) (911,47XL1565B-ANN,1) 

In [6]:
df2 = pd.DataFrame(alarms)
df2.to_csv(path + output_fname, index = False)
print("Processing Time = {} minutes".format((time.time()-start)/60.0))
df2

Processing Time = 2.9562693158785502 minutes


,MachineName,SourceName,Message,Quality,Condition,Mask,NewState,Status,StartTime,EndTime,EndMessage
0,PHD47B,48FIX2040,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IOP-,192,IOP-,193,3,1,2019-03-06 13:19:00,2019-03-06 13:19:02,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IO...
1,PHD47B,48FIX2040,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IOP-,192,IOP-,193,3,1,2019-03-06 13:19:24,2019-03-06 13:19:26,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IO...
2,PHD47B,48FIX2040,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IOP-,192,IOP-,193,3,1,2019-03-06 13:19:35,2019-03-06 13:19:36,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IO...
3,PHD47B,48FIX2040,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IOP-,192,IOP-,193,3,1,2019-03-06 13:19:38,2019-03-06 13:19:39,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IO...
4,PHD47B,48FIX2040,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IOP-,192,IOP-,193,3,1,2019-03-06 13:19:47,2019-03-06 13:19:48,48FIX2040 C206'DAN C4'E A.GAZ CPV = -1 m3/h IO...
...,...,...,...,...,...,...,...,...,...,...,...
2728030,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 6.2 C VEL+,192,VEL+,193,3,1,2019-06-27 18:38:34,2019-06-27 18:38:37,48TIC2040 DESUPERHEATER PV = 7.2 C VEL+ Recover
2728031,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 63.1 C VEL+,192,VEL+,193,3,1,2019-06-27 18:38:59,2019-06-27 18:39:01,48TIC2040 DESUPERHEATER PV = 119.6 C VEL+ Recover
2728032,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 119.6 C IOP,0,IOP,201,3,1,2019-06-27 18:39:02,2019-06-27 18:39:42,48TIC2040 DESUPERHEATER PV = 3.0 C IOP Recover
2728033,PHD47B,48TIC2040,48TIC2040 DESUPERHEATER PV = 4.3 C VEL-,192,VEL-,193,3,1,2019-06-27 18:39:53,2019-06-27 18:39:55,48TIC2040 DESUPERHEATER PV = 6.1 C VEL- Recover
